## User Inputs (Modify as Necessary)

In [106]:
binned_dir = "Directions_Binned" # Name of directory that stores binned results
results_dir = "Results" # Name of directory that will store results
tuning_dir = "Hyperparameters" # Name of directory that stores best hyperparameters
need_tuning = False # Whether or not you want to do hyperparameter tuning (set to False if already done)

folds = 5 # Number of cross-validation folds
ppf = 4 # Number of sample pieces per fold

## Setup

In [107]:
%%bash
python3.11 -m pip install --upgrade pip
pip install numpy
pip install scipy
pip install keras
pip install tensorflow
pip install pandas

In [108]:
import numpy as np
from scipy import io
import pandas as pd
import os
import pickle
from itertools import product
from pprint import pprint

# Import function to get covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

# Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

# Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder


In [109]:
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, binned_dir)
files = next(os.walk(data_dir))[2]

results_dir = os.path.join(parent_dir, results_dir)
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

tuning_dir = os.path.join(parent_dir, tuning_dir)
if not os.path.exists(tuning_dir):
    os.mkdir(tuning_dir)

tuning_files = {'RyFdM1': 'RyFdCon_M1.pickle', 'RyFdS1': 'RyFdCon_S1.pickle',
                'YeFdM1': 'YeFdCon_M1.pickle', 'YeFdS1': 'YeFdCon_S1.pickle',
                'RyDrM1': 'RyDrCon_M1.pickle', 'RyDrS1': 'RyDrCon_S1.pickle',
                'YeDrM1': 'YeDrCon_M1.pickle', 'YeDrS1': 'YeDrCon_S1.pickle'}

datasets = ['RyFdCon_M1', 'RyFdCon_S1', 'RyFdNb_M1', 'RyFdNb_S1',
            'YeFdCon_M1', 'YeFdCon_S1', 'YeFdNb_M1', 'YeFdNb_S1',
            'RyDrCon_M1', 'RyDrCon_S1', 'RyDrNb_M1', 'RyDrNb_S1',
            'YeDrCon_M1', 'YeDrCon_S1', 'YeDrNb_M1', 'YeDrNb_S1']

## Hyperparameter Tuning

In [110]:
if need_tuning:
    best = {}
    for nr in tuning_files:
        best[nr] = {'Avg R2': float('-inf'), 'Epochs': None, 'Bins': None}

    training_range = [0, 0.85]
    testing_range = [0.85, 1]

    bins_before = 1
    bins_current = 1
    bins_after = 1

    bin_combos = [[1, 2], [1, 3], [1, 1], [3, 3], [5, 5]]
    epochs = [10, 20, 50]
    hyperparameters = list(product(epochs, bin_combos))

    for nr in tuning_files:
        print()
        print(tuning_files[nr])
        print()
        for epoch, bins in hyperparameters:
            print('Epoch: ' + str(epoch) + ", Bins: " + str(bins))

            with open(data_dir + "/" + tuning_files[nr], 'rb') as res:
                neural, pos_binned = pickle.load(res, encoding='latin1')

            before = bins[0]
            after = bins[1]

            neural_data = neural[np.any(neural != 0, axis=1)]
            pos_binned = pos_binned[np.any(neural != 0, axis=1)]

            X = get_spikes_with_history(neural_data, before, after, bins_current)
            y = pos_binned

            num_examples = X.shape[0]
            training_set = np.arange(int(np.round(training_range[0] * num_examples)) + before, int(np.round(training_range[1] * num_examples)) - after)
            testing_set = np.arange(int(np.round(testing_range[0] * num_examples)) + before, int(np.round(testing_range[1] * num_examples)) - after)

            # Get training data
            X_train = X[training_set, :, :]
            y_train = y[training_set, :]

            # Get testing data
            X_test = X[testing_set, :, :]
            y_test = y[testing_set, :]

            # Z-score "X" inputs. 
            X_train_mean = np.nanmean(X_train, axis=0)
            X_train_std = np.nanstd(X_train, axis=0)
            X_train = np.divide((X_train - X_train_mean), X_train_std, where=X_train_std != 0)
            X_test = np.divide((X_test - X_train_mean), X_train_std, where=X_train_std != 0)

            # Zero-center outputs
            y_train_mean = np.nanmean(y_train, axis=0)
            y_train = y_train - y_train_mean
            y_test = y_test - y_train_mean

            # Declare model
            model_lstm = LSTMDecoder(units=400, dropout=0, num_epochs=epoch, verbose=0)

            # Fit model
            model_lstm.fit(X_train, y_train)

            # Get predictions
            y_predicted_lstm = model_lstm.predict(X_test)

            # Get results
            R2_lstm = get_rho(y_test, y_predicted_lstm) ** 2
            print('R2:', R2_lstm)

            avg_R2 = np.mean(R2_lstm)
            if avg_R2 > best[nr]['Avg R2']:
                best[nr]['Avg R2'] = avg_R2
                best[nr]['Epochs'] = epoch
                best[nr]['Bins'] = bins
                print(nr + ' Best: ' + str(avg_R2))
    
    pprint(best)
    df = pd.DataFrame(best)
    df.to_csv(tuning_dir + '/Hyperparameters.csv')

In [111]:
params = pd.read_csv(tuning_dir + '/Hyperparameters.csv')
pprint(params)

  Unnamed: 0              RyFdM1               RyFdS1              YeFdM1  \
0     Avg R2  0.9208350861041579  0.28919580860041144  0.8315440080871164   
1     Epochs                  50                   10                  50   
2       Bins              [1, 2]               [5, 5]              [3, 3]   

               YeFdS1              RyDrM1               RyDrS1  \
0  0.6973179015394332  0.9742523580429028  0.24174178088963932   
1                  50                  50                   10   
2              [3, 3]              [1, 2]               [5, 5]   

               YeDrM1              YeDrS1  
0  0.8501741639974295  0.2794369349922743  
1                  10                  10  
2              [1, 3]              [1, 2]  


## LSTM

In [112]:
results = {}

In [150]:
for d in datasets:
    print('Dataset:', d)

    ds, nr = tuple(d.split('_'))
    file = d + '.pickle'
    acc = []
    with open(data_dir + "/" + file, 'rb') as res:
        neural_data, pos_binned = pickle.load(res,encoding='latin1')

    X = get_spikes_with_history(neural_data, bins_before, bins_after, bins_current)
    y = pos_binned
    num_examples = X.shape[0]

    param_key = d[:4] + d.split('_')[1]
    epochs = int(params[param_key][1])
    bins_before = int(params[param_key][2][1])
    bins_current = 1
    bins_after = int(params[param_key][2][4])

    interval_size = 1 / (ppf * folds)
    idx = np.array(list(range(ppf * folds)))
    np.random.shuffle(idx)

    for i in range(folds):
        train_idx = np.sort(np.concatenate((idx[:i * ppf], idx[(i * ppf) + ppf:])))
        test_idx = np.sort(idx[i * ppf:i * ppf + ppf])
        training_set = np.concatenate(tuple([np.arange(int(np.round((train_idx[j] * (1 / (ppf * folds))) * num_examples)) + bins_before, 
                                                       int(np.round(((train_idx[j] * (1 / (ppf * folds))) + 1 / (ppf * folds)) * num_examples)) - bins_after)]
                                                       for j in range(ppf * folds - ppf)), axis=1)
        testing_set = np.concatenate(tuple([np.arange(int(np.round((test_idx[j] * (1 / (ppf * folds))) * num_examples)) + bins_before, 
                                                      int(np.round(((test_idx[j] * (1 / (ppf * folds))) + 1 / (ppf * folds)) * num_examples)) - bins_after)]
                                                      for j in range(ppf)), axis=1)

        # Get training data
        X_train = np.squeeze(X[training_set, :, :])
        y_train = np.squeeze(y[training_set, :]).reshape(-1, 1)

        # Get testing data
        X_test = np.squeeze(X[testing_set, :, :])
        y_test = np.squeeze(y[testing_set, :]).reshape(-1, 1)

        # Z-score "X" inputs. 
        X_train_mean = np.mean(X_train, axis=0)
        X_train_std = np.std(X_train, axis=0)

        # Remove invalid neurons
        zero_std_columns = np.any(X_train_std == 0, axis=0)
        X_train = X_train[:, :, ~zero_std_columns]
        X_train_mean = X_train_mean[:, ~zero_std_columns]
        X_train_std = X_train_std[:, ~zero_std_columns]
        X_test = X_test[:, :, ~zero_std_columns]
        
        X_train = np.nan_to_num(np.divide((X_train - X_train_mean), X_train_std))
        X_test = np.nan_to_num(np.divide((X_test - X_train_mean), X_train_std))

        # Zero-center outputs
        y_train_mean = np.nanmean(y_train, axis=0)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean

        # Declare model
        model_lstm = LSTMDecoder(units=400, dropout=0, num_epochs=epochs, verbose=0)

        # Fit model
        model_lstm.fit(X_train, y_train)

        # Get predictions
        y_predicted_lstm = model_lstm.predict(X_test)

        # Get metric of fit
        rhos_lstm = get_rho(y_test, y_predicted_lstm)
        print('R2:', rhos_lstm ** 2)

        acc.append(rhos_lstm ** 2)

        if np.any(np.isnan(acc)):
            break

    results[d] = np.mean(acc, axis=0)

Dataset: YeDrCon_M1
2/2 [==============================] - 0s 5ms/step
R2: [0.77794057]
2/2 [==============================] - 0s 5ms/step
R2: [0.30796189]
2/2 [==============================] - 0s 4ms/step
R2: [0.50178904]
2/2 [==============================] - 0s 6ms/step
R2: [0.28795954]
2/2 [==============================] - 0s 5ms/step
R2: [0.68429765]


In [151]:
pprint(results)

   Unnamed: 0  RyFdCon_M1  RyFdCon_S1  RyFdNb_M1  RyFdNb_S1  YeFdCon_M1  \
0           0    0.913969    0.246796   0.815952   0.054249    0.795848   

   YeFdCon_S1  YeFdNb_M1  YeFdNb_S1  RyDrCon_M1  RyDrCon_S1  RyDrNb_M1  \
0    0.661796   0.675199   0.292219     0.80316    0.315519   0.431366   

   RyDrNb_S1  YeDrCon_M1  YeDrCon_S1  YeDrNb_M1  YeDrNb_S1  
0    0.10148     0.51199    0.211859   0.438943   0.212154  


In [152]:
df = pd.DataFrame(results)
df.to_csv(results_dir + '/Results.csv')